In [11]:
# Dependencies
from bs4 import BeautifulSoup
import requests, sqlite3, re
import pandas as pd
from geopy.geocoders import Nominatim

# Function to create SQLite table
def create_table():
    conn = sqlite3.connect('gp_data.db')
    c = conn.cursor()
    c.execute("DROP TABLE IF EXISTS gp_info")  # Delete the existing table
    c.execute('''CREATE TABLE gp_info
                 (Name TEXT, Address TEXT, Phone TEXT, 
                 Accepting_New_Patients BOOLEAN, 
                 Accepts_Out_of_Area_Registrations BOOLEAN, 
                 Online_Registration_Available BOOLEAN,
                 Average_Rating REAL)''')
    conn.commit()
    conn.close()

# Function to insert data into SQLite table
def insert_data(gp_data):
    conn = sqlite3.connect('gp_data.db')
    c = conn.cursor()
    for data in gp_data:
        c.execute("INSERT INTO gp_info VALUES (?, ?, ?, ?, ?, ?, ?)",
                  (data['Name'], data['Address'], data['Phone'],
                   data['Accepting New Patients'], 
                   data['Accepts Out of Area Registrations'], 
                   data['Online Registration Available'],
                   data.get('Average Rating', None)))
    conn.commit()
    conn.close()

# Function to scrape data
def scrape_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    gp_blocks = soup.find_all('div', {'class': 'results__details'})
    gp_data = []
    for block in gp_blocks:
        name = block.find('h2', {'id': lambda x: x and x.startswith('orgname_')}).text.strip()
        address = block.find('p', {'id': lambda x: x and x.startswith('address_')}).text.strip()
        phone = block.find('p', {'id': lambda x: x and x.startswith('phone_')}).text.strip()
        tags = block.find_all('strong', {'id': lambda x: x and x.startswith('result_item_')})
        tags_text = [tag.text.strip() for tag in tags]
        gp_data.append({
            'Name': name,
            'Address': address,
            'Phone': phone,
            'Accepting New Patients': 'Accepting new patients' in tags_text,
            'Accepts Out of Area Registrations': 'Accepts out of area registrations' in tags_text,
            'Online Registration Available': 'Online registration available' in tags_text
        })
    return gp_data

# Function to scrape initial GP URLs
def scrape_gp_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    gp_links = []
    for link in soup.find_all('a', {'class': 'nhsapp-open-in-webview'}):
        href = link.get('href')
        if not (href.startswith('javascript') or href.startswith('#')):
            gp_links.append(href)
    return gp_links

# Scrape Reviews Function
def scrape_reviews(gp_links):
    for gp_url in gp_links:
        review_url = f"{gp_url}/ratings-and-reviews"
        try:
            response = requests.get(review_url)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            gp_name = soup.find('h1').text.strip().split('\n')[0].strip()
            review_blocks = soup.find_all('div', {'class': 'org-review'})
            ratings = []
            for block in review_blocks:
                rating_text = block.find('p', {'id': re.compile(r'star-rating-.*')}).text.strip()
                rating_value = float(rating_text.split(' ')[1])
                ratings.append(rating_value)
            if ratings:
                average_rating = sum(ratings) / len(ratings)
                update_db(gp_name, average_rating)
        except requests.HTTPError as e:
            print(f"Could not fetch reviews for {gp_url}: {e}")

# Function to update database
def update_db(gp_name, avg_rating):
    conn = None
    try:
        conn = sqlite3.connect('gp_data.db')
        c = conn.cursor()
        c.execute("UPDATE gp_info SET Average_Rating = ? WHERE Name = ?", (avg_rating, gp_name))
        if c.rowcount == 0:
            print(f"No rows updated. Check if '{gp_name}' exists in the database.")
        conn.commit()
        print(f"Updated {gp_name} with average rating {avg_rating}")
    except sqlite3.Error as e:
        print(f"Database error: {e}")
    finally:
        if conn:
            conn.close()

# Function to export SQLite db table to CSV
def export_table_to_csv(db_path, table_name, csv_path):
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    df.to_csv(csv_path, index=False)
    conn.close()
    print(f"Table {table_name} exported to {csv_path} successfully.")

# Function to create latitude and longitude from addresses
def geocode_address(address):
    geolocator = Nominatim(user_agent="geoapiExercises")
    location = geolocator.geocode(address)
    if location:
        return location.latitude, location.longitude
    else:
        return None, None

# Function to preview the complete table
def preview_complete_table():
    conn = sqlite3.connect('gp_data.db')
    query = "SELECT * FROM gp_info"
    df = pd.read_sql_query(query, conn)
    conn.close()
    num_rows = len(df)
    print(f"Number of rows: {num_rows}")
    print("Complete Table Preview:")
    print(df)

# Main Execution
if __name__ == "__main__":
    # Ask the user for a postcode
    postcode = input("Enter a postcode: ")
    
    # Update the URL with the user's postcode
    url = f"https://www.nhs.uk/service-search/find-a-gp/results/{postcode}"
    
    # Create a new database and table
    create_table()
    
    # Scrape data and insert into the database
    result = scrape_data(url)
    insert_data(result)
    
    # Scrape GP links
    gp_links = scrape_gp_links(url)
    
    # Scrape reviews and update the database
    scrape_reviews(gp_links)
    
    # Export the table to CSV
    export_table_to_csv('gp_data.db', 'gp_info', 'gp_info.csv')

    # Create latitude and longitude from addresses
    geolocator = Nominatim(user_agent="geoapiExercises")
    df = pd.read_csv('gp_info.csv')
    df[['Latitude', 'Longitude']] = df['Address'].apply(lambda x: pd.Series(geocode_address(x)))
    
    # Save the updated DataFrame back to the CSV file
    df.to_csv('gp_info.csv', index=False)
    
    # Preview the complete table
    preview_complete_table()


Updated The Lilyville Surgery with average rating 5.0
Updated Ashville Surgery with average rating 3.1
Updated Cassidy Road Medical Centre with average rating 3.8
Updated The Medical Centre, Dr Jefferies & Partn with average rating 1.1666666666666667
Updated GP at hand with average rating 1.0
Updated Kings Road Medical Centre with average rating 1.5
Updated Earls Court Medical Centre with average rating 1.0
Updated Belgrave Medical Centre with average rating 2.8
Updated Open Door Surgery with average rating 2.0
Updated Heathbridge Practice with average rating 5.0
Updated Putneymead Group Medical Practice with average rating 3.25
Updated Chatfield Health Care with average rating 4.6
Updated North End Medical Centre with average rating 3.625
Updated Chartfield Surgery with average rating 5.0
Updated Wandsworth Medical Centre with average rating 1.8
Updated St Johns Hill Surgery with average rating 1.0
Updated Falcon Road Medical Centre with average rating 4.0
Updated Junction Health Cent

GeocoderInsufficientPrivileges: Non-successful status code 403